In [ ]:
import tensorflow as tf 
from tensorflow.python.compiler.tensorrt import trt_convert as trt
print("\033[92mstart load model\033[0m")
converter = trt.TrtGraphConverterV2(input_saved_model_dir="./model/saved_model")
print("\033[92mload model successfully\033[0m")
converter.convert()
converter.save("./model/saved_model_trt")
tf.config.list_physical_devices()

In [ ]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt
from glob import glob
import numpy as np
import os

# 定义一个函数来读取图像，这个函数需要根据你的实际情况来实现
def load_images(file_name):
    # 这里需要你自己实现如何从file_name读取图像并返回numpy数组
    pass

# 1. 加载已经训练好的TensorFlow模型
saved_model_dir = './model/saved_model' # 请替换为你的模型路径
model = tf.saved_model.load(saved_model_dir)

# 2. 使用TensorRT进行模型优化
params = trt.DEFAULT_TRT_CONVERSION_PARAMS
params = params._replace(max_workspace_size_bytes=(1<<32)) # 修改workspace大小
params = params._replace(precision_mode="FP16") # 设置推理精度
params = params._replace(maximum_cached_engines=100) # 设置最大缓存引擎数量

converter = trt.TrtGraphConverterV2(
    input_saved_model_dir=saved_model_dir,
    conversion_params=params
)
converter.convert()

# 3. 对模型进行序列化并保存
saved_model_dir_trt = 'path_to_save_trt_model' # 请替换为你想要保存TRT模型的路径
converter.save(saved_model_dir_trt)

# 4. 加载优化后的模型进行推理
root = tf.saved_model.load(saved_model_dir_trt)
concrete_func = root.signatures['serving_default']

# 读取输入数据
test_dir = '/path_to_your_test_data' # 请替换为你的测试数据目录
test_data_files = glob(os.path.join(test_dir, '*.png')) # 假设你的测试数据是PNG图像
test_data = [load_images(file) for file in test_data_files]

# 对每一个输入数据进行推理，并保存结果
results = []
for x in test_data:
    # 增加一个维度以匹配模型的输入要求
    x = np.expand_dims(x, axis=0)
    y = concrete_func(x)
    results.append(y)

# 在这里，results列表中包含了所有输入数据的推理结果


In [ ]:
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt
from glob import glob
import numpy as np
import os
from model import *

saved_model_dir = "./model/saved_model"
test_dir = '/simple/low' # 请替换为你的测试数据目录
saved_model_dir_trt = './trt_model' # 请替换为你想要保存TRT模型的路径

# 定义一个函数来读取图像，这个函数需要根据你的实际情况来实现
def load_images(file_name):
    # 这里需要你自己实现如何从file_name读取图像并返回numpy数组
    pass

# 定义一个函数，这个函数将接受所有的输入并返回最后的输出
@tf.function(input_signature=[
    tf.TensorSpec(shape=[None, None, None, 3], dtype=tf.float32),
    tf.TensorSpec(shape=[None, None, None, 3], dtype=tf.float32),
    tf.TensorSpec(shape=[None, None, None, 1], dtype=tf.float32),
    tf.TensorSpec(shape=[None, None, None, 1], dtype=tf.float32),
])
def model_func(input_decom, input_low_i, input_low_i_ratio, input_low_r):
    [R_decom, I_decom] = DecomNet(input_decom)
    output_i, A = Illumination_adjust_curve_net_ratio(input_low_i, input_low_i_ratio)
    output_r = Restoration_net(input_low_r, input_low_i, False)
    return output_i, output_r

# 转换模型
concrete_func = model_func.get_concrete_function()

# 将模型转换为TensorRT模型
params = trt.DEFAULT_TRT_CONVERSION_PARAMS
params = params._replace(max_workspace_size_bytes=(1<<32)) # 修改workspace大小
params = params._replace(precision_mode="FP16") # 设置推理精度
params = params._replace(maximum_cached_engines=100) # 设置最大缓存引擎数量
converter = trt.TrtGraphConverterV2(
    input_saved_model_dir=saved_model_dir,
    conversion_params=params,
    functions=[concrete_func]
)
converter.convert()

# 保存TensorRT模型
converter.save(saved_model_dir_trt)

# 加载优化后的模型进行推理
root = tf.saved_model.load(saved_model_dir_trt)
concrete_func = root.signatures['serving_default']

# 读取输入数据
test_data_files = glob(os.path.join(test_dir, '*.png')) # 假设你的测试数据是PNG图像
test_data = [load_images(file) for file in test_data_files]

# 对每一个输入数据进行推理，并保存结果
results = []
for x in test_data:
    # 增加一个维度以匹配模型的输入要求
    x = np.expand_dims(x, axis=0)
    y = concrete_func(x)
    results.append(y)

# 在这里，results列表中包含了所有输入数据的推理结果
results